## State Formalism for Six Men's Morris Game

In [1]:
from IPython.display import clear_output

import time

def printGameBoard(board):
    
    '''This function prints the current state of the Six Men's Morris board at every point of the gameplay
        
    '''
    
    clear_output()
    
    print("Here is the current state of gameplay")
    
    # Print the current state of the Six Men's Morris game board
    print("    {}-----{}-----{}".format(board[8], board[9], board[10]))
    print("    |     |     |")
    print("    |   {}-{}-{}   |".format(board[0], board[1], board[2]))
    print("    |   |   |   |")
    print("    {}---{}   {}---{} ".format(board[15], board[7], board[3],board[11]))
    print("    |   |   |   |")
    print("    |   {}-{}-{}   |".format(board[6], board[5], board[4]))
    print("    |     |     |")
    print("    {}-----{}-----{}".format(board[14], board[13], board[12]))


In [2]:
initialState = {
    
    'board': [' ' for _ in range(16)],  # A Six Men's Morris board has 16 points or intersections 
    
    #At the beginning of the game, they are all empty
    
    'pieces': {'X': 6, 'O': 6},  # both AI and human player have 6 pieces each yet to be played
    
    'phase': 'placing',  # this basically suggests both players yet have pieces left in hand to play
    
    #'currentPlayer': 'X'  # Indicate the player whose turn it is to play
}


finalState = {
    
    'board': [
        'O', ' ', 'X',
        'X', 'X', ' ',
        'O', ' ', 'X',
        ' ', ' ', ' ',
        ' ', ' ', ' ',' '
    ],  # human player "O" pieces have been reduced to only two pieces on board 
    
    #therefore, the human player can't form a mill or capture the AI's pieces
    
    'pieces': {'X': 0, 'O': 0},  # Both players have finished placing their have been placed, 
    
    'phase': 'moving',  # The game is in a moving phase with both players having no pieces in hand 
    
    'winner': 'X'  # 'X' is the winner
}


#Display initial state of the game on the Six Men's Morris board

printGameBoard(initialState["board"])

print("\n The board is empty in the initial State")

print("\n==================\n")

#Display final state of the game on the Six Men's Morris board

printGameBoard(finalState["board"])

print("\nPlayer O has been reduced to 2 pieces, hence X wins!")

Here is the current state of gameplay
    X----- ----- 
    |     |     |
    |   O- -X   |
    |   |   |   |
     ---    X---  
    |   |   |   |
    |   O- -X   |
    |     |     |
     ----- ----- 

Player O has been reduced to 2 pieces, hence X wins!



## List of Possible Actions and Validation Check

In [3]:
gameBoard=[' ' for _ in range(16)]

player="X"

#create a dictionary that curates the adjacent points for every point on the Six Men's Morris 

adjacentPositions={
    0:[1,7],
    1:[0,2,9],
    2:[1,3],
    3:[2,4,11],
    4:[5,3],
    5:[4,6,13],
    6:[5,7],
    7:[0,6,15],
    8:[9,15],
    9:[1,8,10],
    10:[9,11],
    11:[3,10,12],
    12:[11,13],
    13:[5,12,14],
    14:[13,15],
    15:[7,8,14]
    
}

board_history = []  # this will enable us to track how the Six Men's Morris board states evolve across the game 

import random as rnd

def possibleMovesList(board, player):
    
    """this function checks what moves a player can make given the current state of the Six Men's Morris board"""
    
    list_of_possible_moves = [] #this is a list that curates all possible moves for the designated player
    
    for board_position, piece in enumerate(board):
        
        if piece == player:
            
            for adjacent_piece in adjacentPositions[board_position]:
                
                if board[adjacent_piece] == " ":
                    
                    list_of_possible_moves.append((board_position, adjacent_piece))
                    
    return list_of_possible_moves


def is_valid(board,player, action):
    
    """
    This function checks whether a possible move conforms to the laws of Six Men's Morris
    
    In this game version, pieces can't fly --and can only be moved to vacant adjacent position
    
    """
    
    departure,destination=action
    
    #check if player's piece occupies point of departure
    
    if board[departure]!=player:
        
        return False
    
    #check if the intended point of destination is not occupied already and is contained in the player's adjacent list
    
    if destination not in adjacentPositions[departure] or board[destination]!=" ":
        
        return False
    
    return True






## Evaluation Function for non-terminal state and Utility Function for terminal states

In [4]:
def evalState(board, player):
    
    """
    This function This creates a heuristic scoring system for non-terminal states.
    
    This enables the AI to determine more favorable moves
    
    """
    
    #first establish the list of possible mills which is static and doesn't change at any point of the game
    
    #note that items in this list represent positions where a mill can be formed--NOT the pieces occupying such positions
    
    
    global all_possible_mills
    
    all_possible_mills = [[8, 9, 10], [10, 11, 12], [12, 13, 14], [8, 14, 15], [0, 1, 2], [2, 3, 4], [4, 5, 6], [0, 6, 7]]

    player_mill_number = 0
    
    opponent_mill_number = 0
    
    player_fully_blocked = 0
    
    opponent_fully_blocked = 0
    
    player_mill_potential = 0
    
    opponent_mill_potential = 0

    # Determine the opponent
    
    opponent = "O" if player == "X" else "X"

    # Check for already-formed mills and a player's potential for forming mills
    
    for item in all_possible_mills:
        
        pieces_at_mills = [board[i] for i in item]

        if pieces_at_mills.count(player) == 3: #the player has formed a mill
            
            player_mill_number += 1
            
        if pieces_at_mills.count(opponent) == 3: #opponent has formed a mill
            
            opponent_mill_number += 1

        if pieces_at_mills.count(player) == 2 and pieces_at_mills.count(" ") == 1: #the player is very likely to form a mill
            
            player_mill_potential += 1
            
        if pieces_at_mills.count(opponent) == 2 and pieces_at_mills.count(" ") == 1: #the opponent is very likely to form a mill
            
            opponent_mill_potential += 1

    # Check for the number of blocked pieces
    
    for board_position, piece in enumerate(board):
        
        if piece == player:
            
            adjacent_pieces = adjacentPositions[board_position]
            
            is_blocked = all([board[item] != " " for item in adjacent_pieces]) #True if all adjacent positions are blocked
            
            if is_blocked:
                
                player_fully_blocked += 1

        if piece == opponent:
            
            adjacent_pieces = adjacentPositions[board_position]
            
            is_blocked = all([board[item] != " " for item in adjacent_pieces])
            
            if is_blocked:
                
                opponent_fully_blocked += 1

                
    # To enable the AI player prioritize specific moves, here are modified weights for evaluation factors
    
    mill_weight = 20 #get AI to prioritize forming mills
    
    potential_mill_weight = -10
    
    block_weight = 1  # get AI to win offensively by not excessively defending (and focusing on reducing player's legal moves)
    
    # Now Calculate the evaluation score
    
    mill_score_for_player = (player_mill_number - opponent_mill_number) * mill_weight
    
    #next, get the AI to prioritize blocking opponent's potential mills
    
    potential_mill_score = opponent_mill_potential * potential_mill_weight
    
    block_score = (player_fully_blocked - opponent_fully_blocked) * block_weight

    final_score = mill_score_for_player + potential_mill_score-block_score
    
    return final_score
    


def testWin(board, X_left, O_left, adjacentPositions, board_history):
    
    """This function checks the state of the board if it is a terminal state to return a utility value"""
    
    #X_left is the number of pieces player X has left in hand to play
    
    #O_left is the number of pieces player O has left in hand to play
    
    #adjacentPositions is a dictionary that maps positions of the points (or intersections) in a board to adjacent points
    
    global X_board_count
    
    X_board_count=board.count("X") #this is the number of X pieces on the current board
    
    global O_board_count
    
    O_board_count=board.count("O") #this is the number of O pieces on the current board
    
    #check if player's pieces on the board have been captured and reduced to two after playing all their pieces
    
    if X_board_count<=2 and X_left==0: #Player O has been captured and reduced X pieces on board to two or less
        
        return "O" #player O wins
    
    if O_board_count<=2 and O_left==0: #Player X has been captured and reduced O pieces on board to two or less
        
        return "X" #player X wins 
    
    #check if a player has no legal moves left 
    
    def player_is_blocked(board,adjacentPositions, player):
        
        is_blocked=True
        
        for piece_position,piece in enumerate(board):
            
            if piece==player:
                
                adjacentPieces=adjacentPositions[piece_position]
                
                if any([board[adjacent]==" " for adjacent in adjacentPieces]):
                    
                    is_blocked=False #empty position found adjacent to this player
                    
                    break
                    
        return is_blocked
    
    if player_is_blocked(board,adjacentPositions, "X") and X_left==0:
        
        return "O" #Player O has won
    
    if player_is_blocked(board,adjacentPositions, "O") and O_left==0:
        
        return "X" #Player X has won
    
    # Classify game as a Draw if a significant number of moves have been unduly repeated 
    
    #This is when players have finished placing their pieces
    
    if board_history.count(tuple(board)) >= 4 and X_left==0 and O_left==0:
        
        return "Draw"
    
    
    return "Keep Playing" #there is no win with both players having no pieces left to play. Hence it is a draw



## Alpha-Beta Algorithm allowing the agent to play the game

In [5]:
import math

import copy

    
def minimaxDecision(currentBoard,X_left,O_left,depth=5):
    
    """this function decides which move the AI player should choose at every state of the game 
    
       to maximize its chances of winning
    
    """
    best_move=None #this ultimately the move the AI will choose after evaluating all possible moves or piece placements
    
    beta=math.inf
    
    best_value=-math.inf #this represents the best option the AI has seen so far
    
    #this will act as alpha for the AI to use when picking its best moves. 
    
    #It gets updated every time we find a better score
    
    maxDepth=5
    
    if X_left>0: #game is in phase where the AI is yet to finish placing its pieces
            
        for i in range(len(currentBoard)):
            
            if currentBoard[i]==" ": #there is a vacant position
                
                currentBoard[i]="X" #the AI places its piece in that vacant position
                
                val,_=minValue(currentBoard,best_value,beta, depth-1,X_left-1, O_left)
                
                 #(currentBoard, alpha, beta, depth, X_left, O_left)
                
                if val>best_value: #found a better option with a higher evaluation
                    
                    best_value=val
                    
                    best_move=i
                    
                currentBoard[i]=" " #here we undo the move since we didn't use a deepcopy of currentBoard 
            
                    
    else: #game in movement phase with AI having no pieces left in hand to play
        
        actions=possibleMovesList(currentBoard,"X") #generates a list of tuple containing where X piece is and  
        
        # possible vacant positions where the X piece can move to
        
        for action in actions:
            
            currentBoard[action[0]],currentBoard[action[1]]=" ", "X" #we move X piece to the next adjacent vacant position
            
            val,_=minValue(currentBoard,best_value, beta, depth-1, X_left, O_left)
           
            if val>best_value:
                
                best_value=val
                
                best_move=action
                
            currentBoard[action[0]],currentBoard[action[1]]="X", " " #undoing the move
                
    return best_move         


import copy

def maxValue(currentBoard,alpha,beta,depth, X_left, O_left):
    
    #check if a terminal state is reached and return utility value
    
    whoWon=testWin(currentBoard,X_left, O_left, adjacentPositions,board_history)
    
    if whoWon=="X":
        
        return 10,None # Prioritizing quicker wins.The None here means there is no best move available being a terminal state
    
    if whoWon=="O":
        
        return -10,None #preferring delayed losses
    
    if whoWon=="Draw":
        
        return 0,None
    
    if depth==0:
        
        eval=evalState(currentBoard, "X")
        
        return eval,None
    
    
    maxVal=-math.inf
    
    best_move=None
    
    #First, consider if the AI still has pieces left in hand to place on the board
    
    if X_left>0:
        
        for i in range(len(currentBoard)):
            
            if currentBoard[i]==" ": #vacant point to place piece
                
                nextBoard=copy.deepcopy(currentBoard)
                
                nextBoard[i]="X"
                
                value,_=minValue(nextBoard,alpha,beta,depth-1,X_left-1,O_left)
                
                if value>maxVal:
                    
                    maxVal=value
                    
                    best_move=i
                    
                    if maxVal>=beta:
                        
                        return value,best_move
                    
                    alpha=max(alpha,value)
                        
    else: #player X has finished placing all its pieces and can only move pieces on board   
        
        actions=possibleMovesList(currentBoard, "X")
        
        for action in actions:
            
            nextBoard=copy.deepcopy(currentBoard)
            
            nextBoard[action[0]], nextBoard[action[1]]=" ", "X"
            
            value,_=minValue(nextBoard, alpha, beta, depth-1, X_left, O_left)
            
            if value>maxVal:
                
                maxVal=value
                
                best_move=action
                
            if maxVal>=beta:
                
                return maxVal,best_move
            
            alpha=max(alpha,value)
            
    return maxVal, best_move
            
    

def minValue(currentBoard, alpha, beta, depth, X_left, O_left):
    
    #check if a terminal state is reached and return utility value
    
    whoWon=testWin(currentBoard,X_left, O_left, adjacentPositions,board_history)
    
    if whoWon=="X":
        
        return 10,None 
    
    if whoWon=="O":
        
        return -10,None 
    
    if whoWon=="Draw":
        
        return 0,None
    
    if depth==0:
        
        eval=evalState(currentBoard, "X")
        
        return eval,None
    

    minVal=math.inf
    
    best_move = None
    
    #as before, we start with the phase where minimizing player still has pieces in hand to place on the board
    
    
    if O_left > 0:  # indicate that gameplay is in the placing phase
        
        for i in range(len(currentBoard)):
            
            if currentBoard[i]==" ": #vacant point to place piece
                
                nextBoard=copy.deepcopy(currentBoard)
                
                nextBoard[i]= "O" #placing O piece
                
                value,_=maxValue(nextBoard,alpha, beta, depth-1, X_left,O_left-1)
                
                if value<minVal:
                    
                    minVal=value
                    
                    best_move=i
                    
                if minVal<=alpha:
                    
                    return minVal, best_move
                
                beta=min(beta,minVal)
                    
    else: #minimizing player has placed all pieces and can only move pieces on the board
        
        actions=possibleMovesList(currentBoard, "O")
        
        for action in actions:
        
            nextBoard=copy.deepcopy(currentBoard)

            nextBoard[action[0]],nextBoard[action[1]]=" ", "O"
            
            value,_=maxValue(nextBoard,alpha, beta, depth-1, X_left,O_left-1)
                
            if value<minVal:
                    
                    minVal=value
                    
                    best_move=action
                    
            if minVal<=alpha:
                    
                    return minVal, best_move
                
            beta=min(beta,minVal)
            
    return minVal,best_move
        




## Gameloop and associated functions for AI agent to play against a human player

In [6]:
def printReference(board):
    
    
    '''This function prints the board with each position labelled with the board index
        
       This way, the human player knows the board index of the position he desires to place (or move) a piece

    '''
    
    clear_output()
    
    print("Here is the visualized board to help you know the index position of the points on the Six Men's Morris")
    
    # Print the current state of the Six Men's Morris game board
    print("    {}-----{}-----{}".format(board[8], board[9], board[10]))
    print("    |     |     |")
    print("    |   {}-{}-{}   |".format(board[0], board[1], board[2]))
    print("    |   |   |   |")
    print("   {}---{}   {}---{} ".format(board[15], board[7], board[3],board[11]))
    print("    |   |   |   |")
    print("    |   {}-{}-{}   |".format(board[6], board[5], board[4]))
    print("    |     |     |")
    print("   {}-----{}----{}".format(board[14], board[13], board[12]))

reference=[i for i in range(16)]

In [7]:
def switchPlayer():
    
  """This function basically switches turns between both players"""

  global PlayerTurn

  if PlayerTurn == 'X':
        
    PlayerTurn = 'O'
    
  else:
    
    PlayerTurn = 'X'
    

def capturePiece(board, opponent):
    
    """
        This function enables the player to capture its opponent's pieces when a mill is formed
        
        An opponent's piece can only be captured:
        
        i.When it is not part of a mill
        
        ii.When all the remaining pieces of the opponent are in a mill formation
        
    """
    
    global X_board_count, O_board_count
    
    all_possible_mills=[[8,9,10],[10,11,12],[12,13,14],[8,14,15],[0,1,2],[2,3,4],[4,5,6],[0,6,7]]

    def isMillFormed(board, player, point):
        
        """ Check if placing or moving to the designated point creates a mill """
        
        for mill in all_possible_mills:
            
            if point in mill and all(board[a] == player for a in mill):
                
                return True
            
        return False

    def opposingPlayerMillPieces(board, opposingPlayer):
        
        """ Curate all mill positions occupied by the opposing player """
        
        opposing_player_mills = []
        
        for b in all_possible_mills:
            
            if all(board[pos] == opposingPlayer for pos in b):
                
                opposing_player_mills.extend(b)  # Here, we use extend to add positions
                
        return opposing_player_mills

    def determineStrategicCapture(board, opponent):
        
        """ Rather than randomly choose pieces to capture, the AI should take a strategic approach
        
            This should make the AI more competitive against the human player
        
        """
        possible_captures = [pos for pos, piece in enumerate(board) if piece == opponent and not isMillFormed(board, opponent, pos)]
        
        for pos in possible_captures:
            
            for mill in all_possible_mills:
                
                
                if pos in mill and board[mill[0]] == board[mill[1]] == opponent and board[mill[2]] == ' ':
                    
                    return pos  # Capture piece to disrupt human player's capacity to form a mill later
                
        return rnd.choice(possible_captures) if possible_captures else None

    # Next, capture a piece when a mill is formed
    
    while True:
        
        try:
            
            if opponent == "O":  # The AI will now capture an O piece
                
                captured_piece = determineStrategicCapture(board, "O")
                
                printGameBoard(board)
                
                print("\nOops, your piece has been captured!!!")
                
                time.sleep(3)
                
                if captured_piece is None:
                    
                    break  # the AI couldn't find a valid capture

            else:  # Human player's turn
                
                captured_piece = int(input("Congrats,you have formed a mill!Enter the opponent's piece position to capture (0-15): "))

            # Ensure the captured piece aligns with the Six Men's Morris rules for capturing pieces
            
            #Ensure chosen piece position is within the board
            
            opponent_piece_positions = [pos for pos, piece in enumerate(board) if piece == opponent]
            
            all_in_mill = all(isMillFormed(board, opponent, pos) for pos in opponent_piece_positions)

            if all_in_mill or (0 <= captured_piece <= 15 and board[captured_piece] == opponent and not isMillFormed(board, opponent, captured_piece)):
                
                if board[captured_piece] == "X":
                    
                    X_board_count -= 1  # reduce the number of X pieces left on the board after capture
                    
                else:
                    
                    O_board_count -= 1  # reduce the number of O pieces left on the board after capture

                board[captured_piece] = ' ' #removing captured piece from board
                
                break
                
            else:
                
                print("Invalid capture. Choose an opponent's piece either in a valid position")
                
        except ValueError:
            
            print("Invalid input. Please enter a number between 0 and 15.")


In [8]:
def isMillFormed(board, player, point):
                
        
        for mill in all_possible_mills:
            
            if point in mill and all(board[a] == player for a in mill):
                
                return True
            
        return False

In [ ]:
def adjacentMove(board,destination): #check if position is adjacent to 0's piece
                    
    for pos,player in enumerate(board):
                        
        found=False

        if player=="O":

            adjacentO=adjacentPositions[pos]

            if destination in adjacentO and board[destination]==" ":

                found=True

                break
                                
    return found    
    

PlayerTurn = 'X'

def sixMenMorrisLoop():
    
    """
        This function will enable the AI to play the Six Men's Morris against the human player
    """
    
    # First, initialize the state of the Six Men's Morris board
    
    board=[' ' for _ in range(16)]
    
    X_left, O_left = 6, 6  # both the AI and the minimizing human player start the game with 6 pieces
    
    #PlayerTurn = "X"  # AI starts the game
    
    while True:
        
        #printGameBoard(board)
        
        #time.sleep(3)
        
        if PlayerTurn == "X":
            
            # It is the AI player's turn
            
            action = minimaxDecision(board, X_left, O_left)
            
            if X_left > 0: #here the game is still in the piece placement phase
                
                board[action] = 'X'
                
                X_left -= 1
                
                if isMillFormed(board, "X", action):
                    
                    capturePiece(board, "O")
            
                
            else: #here the game is now in the piece movement phase
                
                board[action[0]], board[action[1]] = ' ', 'X'
                
                if isMillFormed(board, "X", action[1]):
                    
                    capturePiece(board, "O")
                 
                
                
        else:
            # Human player's turn
            
            reference=[i for i in range(16)]
            
            printReference(reference) #print board reference so they can understand point mapping
            
            time.sleep(3) # Give the human player some time to see and understand the reference board before clearing 
            
            printGameBoard(board)
            
            if O_left > 0: #here the game is still in the piece placement phase
                
                print(f"You have {O_left} pieces in hand to place on the board")
                
                move= int(input("Choose your preferred point on the board to place your piece"))
            
                while(board[move]!=" " or move<0 or move >15):
                
                    move=int(input("Please choose a valid point to place your piece"))
                
                board[move] = 'O'
                
                O_left -= 1
                
                if isMillFormed(board, "O", move):
                    
                    capturePiece(board, "X")
                    

                
            else: #here the game is now in the piece movement phase
                                
                departure= int(input("choose the position of the piece you want to move on the board"))
                
                #Ensure that the desired destination is vacant and the index is within the board
            
                while(board[departure]==" " or departure<0 or departure >15 or board[departure]!="O" ):
                    
                    departure=int(input("please choose a valid position"))
                
                destination= int(input("choose the position you want to move your piece to on the board"))
                
                #Ensure move is valid and aligns with Six Men's Morris rules
                
                while not (0 <= destination <= 15 and board[destination] == " " and is_valid(board, "O", (departure, destination))):
                    
                    
                    destination = int(input("Please, you need to choose a valid adjacent position to move your piece: "))
                

                board[departure], board[destination] = ' ', 'O'
                
                if isMillFormed(board, "O", destination):
                    
                    capturePiece(board, "X")
                
        
        board_history.append(tuple(board)) # Here, add the current board state to history 
        
        #we will later use this board history to track repetitions when determining draws
        
        
        # Check for win or draw
        
        result = testWin(board, X_left, O_left, adjacentPositions, board_history)
        
        
        if result in ["X", "O"]: #checking if either the AI or the human player wins
            
            printGameBoard(board)
            
            print(f"The game has ended! The winner is: {result}")
            
            break
            
        elif result=="Draw": #checking if is a draw with no clear winner
            
            printGameBoard(board)
            
            print ("The game is over! It is a draw")
            
            break
            
        elif result=="keep playing": #the game has not ended so, play should continue
            
            pass
        
        switchPlayer()


# Start the game

sixMenMorrisLoop()

Here is the current state of gameplay
     ----- ----- 
    |     |     |
    |   X- -    |
    |   |   |   |
     ---     ---  
    |   |   |   |
    |    - -    |
    |     |     |
     ----- ----- 
You have 6 pieces in hand to place on the board




# MONTE CARLO TREE SEARCH IMPLEMENTATION

In [ ]:
statesEvaluation={} #stores states to be evaluated

statesToBeUpdated=[] #stores states to be updated in statesEvaluation during backpropagation

def addToStateEvaluation(board):
    
    
    #add a board state to the statesEvaluation dictionary
    boardStr=""
    
    for piece in board:
        
        boardStr+=str(piece)
    
    if boardStr in statesEvaluation:
        
        pass
    
    else:
        
        statesEvaluation[boardStr]=[0,0]
        


def checkIfBoardInStateEvaluation(board):
    
    #check if a board state is in the statesEvaluation dictionary
    
    boardStr=""
    
    for piece in board:
        
        boardStr+=str(piece)
    
    if boardStr in statesEvaluation:
        
        return statesEvaluation[boardStr]
    
    else:
        
        return False
    
def addTostatesToBeUpdated(board):
    
    """This function will record states in the State Evaluation dictionary to updated later during the backpropagtion """
    
    boardStr=""
    
    for piece in board:
        
        boardStr+=str(piece)
    
    if boardStr in statesToBeUpdated:
        
        pass
    
    else:
        
        statesToBeUpdated.append(boardStr)

In [ ]:
winningTerminalValue = 20

loosingTerminalValue = -15

drawTerminalValue = 0

learningRateC = 2 # this is the C parameter

AITime = 10

def selectionPart(curr):
    
    global statesToBeUpdInEvalDict
    
    statesToBeUpdInEvalDict = []
    
    playerID="X"
    
    current=[curr[i] for i in range(16)] #create a deep copy from curr
    
    while(True):
    
        maxUCB = -math.inf 
        
        maxUCBState = []
        
        minUCB = math.inf
        
        minUCBState = []
        
        cMove = -1
    
        currentBoardEvaluation=checkIfBoardInStateEvaluation(current) #
        
        addTostatesToBeUpdated(current)
        
        #check for all possible moves
        
        nextState=[current[a] for a in range(16)]
        
        if X_left>0: #pieces left for player X to play on the board
            
            nextState=[current[a] for a in range(16)]
            
            for board_position in range(len(nextState)):
                
                if nextState[board_position]==" ":
                    
                    nextState[board_position]="X"
                    
                    nextBoardEvaluation=checkIfBoardInStateEvaluation(nextState)

                    if nextBoardEvaluation==False or nextBoardEvaluation[1]==0:

                        checkEndGameResult=checkEndGame(nextState)

                        if checkEndGameResult!=False: #terminal node

                            return (nextState,checkEndGameResult, True)
                        
                        else:

                            return((nextState,playerID, False))
                        
                    
                # if not a leaf node, we hve to calculate its UCB. This will enable us to pick the best state
                
                # boardUCB = vi + C * sqrt ( ln (N) / ni
                
                vi=nextBoardEvaluation[0]/nextStateEvaluation[1]
                
                N=currentBoardEvaluation[1]
                
                ni=nextBoardEvaluation[1]
                
                boardUCB=vi+ 2*(sqrt(ln(N)/ni))
                
                if playerID=="X": #maximizing player
                    
                    if boardUCB>maxUCB:
                        
                        maxUCB=boardUCB
                        
                        cMove=board_position
                        
                        maxUCBState=[nextState[s] for s in range(16)]
                        
                else: #minimizing player
                    
                    if boardUCB<minUCB:
                        
                        minUCB=boardUCB
                        
                        cMove=board_position
                        
                        minUCBState=[nextState[s] for s in range(16)]
                        
        # pick the maxUCB and move forward with that state
        # so now current state is the maxUCB state for the Max player or minUCB state for the min Player            
                        
            if playerID=="X":
                
                currentBoard=[maxUCBState[a] for a in range(16)]
                
                checkEndGameResult=checkEndGame(currentBoard)
                
                if checkEndGameResult!=False:
                    
                    return (currentBoard,checkEndGameResult, True)
                
                playerID="O"
                
            else: #minimizing player
                
                currentBoard=[minUCBState[a] for a in range(16)]
                
                checkEndGameResult=checkEndGame(currentBoard)
                
                if checkEndGameResult!=False:
                    
                    return (currentBoard,checkEndGameResult, True)
                
                playerID="X"
                
                    
                    
              
        else: #player X has no piece in hand to play
            
            actions=possibleActions(current,playerID)
                
            for action in actions:
                
                if isValid(board, action,player):
                
                    departure,destination=action
                
                    nextState[departure],nextState[destination]=" ","X"
                    
                    nextStateEvaluation=checkIfBoardInStateEvaluation(nextState)

                    if nextStateEvaluation==False or nextStateEvaluation[1]==0:

                        checkEndGameResult=checkEndGame(nextState)

                        if nextStateStatus!=False: #terminal node

                            return (nextState,checkEndGameResult, True)
                        
                        else:

                            return((nextState,playerID, False))
                        
                
                    # if not a leaf node, so calculate its UCB so we can choose the best state
                    # boardUCB = vi + 2 * sqrt ( ln (N) / ni

                    vi=nextBoardEvaluation[0]/nextStateEvaluation[1]

                    N=currentBoardEvaluation[1]

                    ni=nextBoardEvaluation[1]

                    boardUCB=vi+ 2*(sqrt(ln(N)/ni))

                    if playerID=="X": #maximizing player

                        if boardUCB>maxUCB:

                            maxUCB=boardUCB

                            cMove=action

                            maxUCBState=[nextState[s] for s in range(16)]

                    else:

                        if boardUCB<minUCB:

                            minUCB=boardUCB

                            cMove=board_position

                            minUCBState=[nextState[s] for s in range(16)]        

            #start
            
            # pick the maxUCB and move forward with that state
            # so now current state is the maxUCB state for the Max player or minUCB state for the min Player            
                        
            if playerID=="X":
                
                currentBoard=[maxUCBState[a] for a in range(16)]
                
                checkEndGameResult=checkEndGame(currentBoard)
                
                if checkEndGameResult!=False:
                    
                    return (currentBoard,checkEndGameResult, True)
                
                playerID="O"
                
            else: #minimizing player
                
                currentBoard=[minUCBState[a] for a in range(16)]
                
                checkEndGameResult=checkEndGame(currentBoard)
                
                if checkEndGameResult!=False:
                    
                    return (currentBoard,checkEndGameResult, True)
                
                playerID="X"
        

In [ ]:
def playout(curr,playerID):
    
    currentBoard=[curr[i] for i in range(16)]
    
    while(True):
        
        
        if X_left>0:
        
            possibleActions=[]

            for board_position in range(len(currentBoard)):

                if currentBoard[board_position]==" ":

                    possibleActions.append(board_position)

            cMove=int(rnd.random()*len(possibleActions))

            currentBoard[cMove]=playerID

            result=testWin(currentBoard, X_left, O_left, adjacentPositions, board_history) 

            if result=="X":

                return winningTerminalValue

            if result=="O":

                return loosingTerminalValue 

            if result=="Draw":

                return drawTerminalValue

             # if not a terminal state, switch player

            if (playerID == 'X'):
                playerID = 'O'
            else:
                playerID = 'X'
            
        else: #X has finished pieces

            possibleMoves=possibleActions(currentBoard, playerID)

            c=int(rnd.random()*len(possibleMoves))

            randomMove=possibleMoves[c]

            while(isValid(board,randomMove,playerID)==False):

                c=int(rnd.random()*len(possibleMoves))

                randomMove=possibleMoves[c]

            departure,destination=randomMove

            currentBoard[departure],currentBoard[destination]=" ","X"

            result=testWin(currentBoard, X_left, O_left, adjacentPositions, board_history) 

            if result=="X":

                return winningTerminalValue

            if result=="O":

                return loosingTerminalValue 

            if result=="Draw":

                return drawTerminalValue

             # if not a terminal state, switch player

            if (playerID == 'X'):
                playerID = 'O'
            else:
                playerID = 'X'

In [ ]:
def monteCarloTreeSearch (cur):
    
    startTime=time.time()
    
    if checkIfBoardInStateEvaluation(cur)==False:
        
        addToStateEvaluation(cur)
        
    while (True):
        
        if ((time.time() - startTime) > AITime): 
            
            break
        
        stateToExplore, PlayerID, endGame = selectionPart(cur)
        
        if endGame==False:
            
            addToStateEvaluation(stateToExplore)
            
            addTostatesToBeUpdated(stateToExplore)
           
            returnedUtility = playout (stateToExplore, PlayerID)
            
        else: #terminal state reached
            
            # check which player won and return utility
            
            if PlayerID == 'X':
                
                returnedUtility = winningTerminalValue
                
            elif PlayerID == 'O':
                
                returnedUtility = loosingTerminalValue
                
            else:
                returnedUtility = drawTerminalValue
        
        #Start the backpropagation
        
        for eachBoardState in statesToBeUpdated:
            
            currentBoardEvaluation=statesEvaluation[eachBoardState]
            
            currentBoardEvaluation[0]+=returnedUtility
            
            currentBoardEvaluation[1]+=1
            
            statesEvaluation[eachBoardState]=currentBoardEvaluation
        
    
    # when we stop, we choose the best action so far
    
    maxUCB = -math.inf
    
    cMove = -1
    
    # save the eval and nbTimes of current or parent state
    
    currentBoardEvaluation= checkIfBoardInStateEvaluation(cur)
    
    
    if X_left>0:
        
            nextState=[current[a] for a in range(16)]
            
            for board_position in nextState:
                
                if nextState[board_position]==" ":
                    
                    nextState[board_position]="X"
                    
                    nextStateEvaluation=checkIfBoardInStateEvaluation(nextState)

                    vi=nextStateEvaluation[0]/nextStateEvaluation[1]

                    N=currentBoardEvaluation[1]

                    ni=nextBoardEvaluation[1]

                    boardUCB=vi+ 2*(sqrt(ln(N)/ni))
                    
                
                    if playerID=="X": #maximizing player
                    
                        if boardUCB>maxUCB:

                            maxUCB=boardUCB

                            cMove=board_position
                    else:
                        
                        pass
    
    else: #no more pieces in hand to play
     
            actions=possibleActions(current,playerID)
                
            for action in actions:
                
                if isValid(board, action,player):
                
                    departure,destination=action
                
                    nextState[departure],nextState[destination]=" ","X"
                    
                    nextStateEvaluation=checkIfBoardInStateEvaluation(nextState)
             
                    vi=nextBoardEvaluation[0]/nextStateEvaluation[1]

                    N=currentBoardEvaluation[1]

                    ni=nextBoardEvaluation[1]

                    boardUCB=vi+ 2*(sqrt(ln(N)/ni))

                    if playerID=="X": #maximizing player

                        if boardUCB>maxUCB:

                            maxUCB=boardUCB

                            cMove=action
    
    return cMove
